# QUIZ - 2024-08

In [36]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [37]:
from service.models import Service, ServiceCategory, ServiceProvider, Customer, Appointment
from datetime import date, time, datetime
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan
from django.db.models.functions import Concat
from django.db.models import CharField, Value as V

## PART 1 - CRUD Operations
1.1 สร้างบริการ (`Service`) รายละเอียดตามด้านล่าง และเพิ่มเข้าไปในหมวดหมู่ (`ServiceCategory`) ชื่อ "Hair Care"

- Name: "Hair Shower"
- Desciption: "Cold hair shower"
- Price: 100 (Provider 1) และ 200 (Provider 2)
- Service Provider: "Provider 1" และ "Provider 2"

*Hint: ดังนั้นต้องสร้าง 2 แถวในตาราง `Service`*


(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [60]:
#     service_provider = models.ForeignKey(ServiceProvider, on_delete=models.CASCADE)
#     name = models.CharField(max_length=100)
#     description = models.TextField(blank=True)
#     price = models.PositiveIntegerField(default=0)

pro1 = ServiceProvider.objects.get(name = "Provider 1")
pro2 = ServiceProvider.objects.get(name = "Provider 2")

ser_pro1 = Service.objects.create(
    name = "Hair Shower",
    description = "Cold hair shower",
    price = 100,
    service_provider = pro1)

ser_pro2 = Service.objects.create(
    name = "Hair Shower",
    description = "Cold hair shower",
    price = 200,
    service_provider = pro2)

In [61]:
# Check result
cat = ServiceCategory.objects.first()
cat.services.count()

4

1.2 ทำการแก้ไขราคาของ "Hair Shower" ของ "Provider 1" จาก 100 เป็น 300

(0.25 คะแนน)

In [62]:
ser_pro1.price = 300
ser_pro1.save()

In [63]:
# Check result
services = Service.objects.filter(name="Hair Shower")
for service in services:
    print(f"Service Name: {service.name}, Provider Name: {service.service_provider.name}, Price: {service.price}")

Service Name: Hair Shower, Provider Name: Provider 2, Price: 200
Service Name: Hair Shower, Provider Name: Provider 1, Price: 300


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-15 โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Massage" ของ "Provider 3" เป็น "Hair Shower" ของ "Provider 1" แทน และทำการเลื่อนวันที่นัดหมายจาก 2024-08-15 เป็น 2024-08-18

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [81]:
pro1 = ServiceProvider.objects.get(name = "Provider 1")
pro3 = ServiceProvider.objects.get(name = "Provider 3")
ser_old = Service.objects.get(name = "Massage", service_provider__name = "Provider 3")
ser_new = Service.objects.get(name = "Hair Shower", service_provider__name = "Provider 1")
old_app = Appointment.objects.get(appointment_date = '2024-08-15')

# appointment_date, appointment_time, created_at, customer, customer_id, id, service, service_id
new_app = Appointment.objects.get(service__name = pro1 , service = ser_old)
new_app.service = ser_new
new_app.appointment_date = old_app
new_app.save()

DoesNotExist: Appointment matching query does not exist.

In [ ]:
# Check result
res = Appointment.objects.order_by("-appointment_date").first()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

1.4 ทำตามขั้นตอนดังนี้

1. สร้างผู้ให้บริการ (`ServiceProvider`) ชื่อ "Super Provider" ขึ้นมา (email: super@gmail.com, phone: 089-999-9999) และทำการเพิ่มบริการ (`Service`) ให้กับผู้ให้บริการนี้ (เทพมากสามารถทำได้ทุกบริการ) โดยทุกบริการราคาเท่ากับ 500 บาท

(1 คะแนน)

In [64]:
# สร้างผู้ให้บริการ
new_ser = ServiceProvider.objects.create(
    name = "Super Provider",
    email = "super@gmail.com",
    phone = "089-999-9999")

In [ ]:
# เพิ่มบริการ (Service) ให้กับผู้ให้บริการนี้


In [ ]:
# Check result
for s in Service.objects.filter(service_provider__name="Super Provider"):
    print(f"Service Name: {s.name}, Price: {s.price}, Provider: {s.service_provider.name}")

## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่นัดหมาย (`Appointment.appointment_date`) ในช่วงวันที่ 2024-08-05 ถึง 2024-08-10 ของ "Service Provider 1"

**Expected Output**

จะต้องมี 3 รายการ

(0.5 คะแนน)

In [16]:
appointment = Service.objects.filter(service_provider__name = "Provider 1", appointment__appointment_date__gte = "2024-08-05", appointment__appointment_date__lte = "2024-08-10").count()
appointment

5

2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Manicure, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2
```

In [25]:
app = Appointment.objects.filter(created_at__gte = "2024-08-01", created_at__lte = "2024-08-02")

for i in app:
    print(f"Name: {i.service.name}, Provider: {i.service.service_provider.name}")

Name: Haircut, Provider: Provider 1
Name: Massage, Provider: Provider 1


2.3 ให้แสดงผลข้อมูลลูกค้า (`Customer`) ดังในตัวอย่าง

```python
[
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [30]:
cus = Customer.objects.annotate(name =  )
for i in cuss:
    print(json.dumps(i, indent=4, sort_keys=False))

NameError: name 'name' is not defined

2.4 ให้แสดงข้อมูลผู้ให้บริการ (`ServiceProvider`) ดังนี้

*หมายเหตุ: Appointment Num หมายถึงจำนวนนัดหมายที่ผู้ให้บริการนั้นมีการได้รับนัด และ Appointed Service หมายถึงบริการที่มีการนัดหมายของผู้ให้บริการนั้นๆ*

(1 คะแนน)

**Expected Output**

```python
Name: Provider 1, Appointment Num: 7, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure, Massage, Massage
Name: Provider 2, Appointment Num: 5, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure
Name: Provider 3, Appointment Num: 2, Appointed Services: Manicure, Massage
Name: Provider 4, Appointment Num: 1, Appointed Services: Haircut
```

In [55]:
s = Appointment.objects.annotate(AppointmentNum = Count('customer_id'))
for i in s:
    print(f"Name: {i.service_provider.name}")

FieldError: Cannot resolve keyword 'customer_id' into field. Choices are: appointment, description, id, name, price, service_provider, service_provider_id, servicecategory

2.5 ให้หาว่าบริการ (`Service`) ไหนของผู้บริการไหนได้รับความนิยมสูงสุด (มีจำนวนนัดหมายมากที่สุด) และ นิยมน้อยที่สุด (มีจำนวนนัดหมายน้อยที่สุด)

(0.5 คะแนน)

**Expected Output**

```python
Name: Manicure, Provider: Provider 1, Appointment Num: 3
Name: Massage, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 2, Appointment Num: 3
Name: Massage, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 1, Appointment Num: 1
Name: Massage, Provider: Provider 2, Appointment Num: 1
Name: Manicure, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 4, Appointment Num: 1
Name: Haircut, Provider: Provider 2, Appointment Num: 1
Name: Massage, Provider: Provider 4, Appointment Num: 0
Name: Manicure, Provider: Provider 4, Appointment Num: 0
Name: Haircut, Provider: Provider 3, Appointment Num: 0
```

In [ ]:
obj = Service.objects.aggregate(Count())

2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อบริการ (`Service.name`) เป็นตัวใหญ่ (uppercase) และราคา (`Service.price`) เป็น 2 เท่า และเรียงลำดับตามราคา

(0.5 คะแนน)

**Expected Output**

```python
Name: HAIRCUT, Provider: Provider 1, New Price: $40
Name: MANICURE, Provider: Provider 1, New Price: $60
Name: MASSAGE, Provider: Provider 1, New Price: $200
Name: HAIRCUT, Provider: Provider 2, New Price: $40
Name: MANICURE, Provider: Provider 2, New Price: $60
Name: MASSAGE, Provider: Provider 2, New Price: $200
Name: HAIRCUT, Provider: Provider 3, New Price: $40
Name: MANICURE, Provider: Provider 3, New Price: $60
Name: MASSAGE, Provider: Provider 3, New Price: $200
Name: HAIRCUT, Provider: Provider 4, New Price: $40
Name: MANICURE, Provider: Provider 4, New Price: $60
Name: MASSAGE, Provider: Provider 4, New Price: $200
```

In [50]:
#Entry.objects.filter(number_of_comments__gt=F("number_of_pingbacks") * 2)
service = Service.objects.annotate(upper = Upper('name')).all().order_by('price')
for i in service:
    print("Name: %s, Provider: %s, New Price: $%d" %(i.upper, i.service_provider.name, i.price))

Name: HAIRCUT, Provider: Provider 1, New Price: $20
Name: HAIRCUT, Provider: Provider 2, New Price: $20
Name: HAIRCUT, Provider: Provider 3, New Price: $20
Name: HAIRCUT, Provider: Provider 4, New Price: $20
Name: MANICURE, Provider: Provider 4, New Price: $30
Name: MANICURE, Provider: Provider 1, New Price: $30
Name: MANICURE, Provider: Provider 3, New Price: $30
Name: MANICURE, Provider: Provider 2, New Price: $30
Name: MASSAGE, Provider: Provider 4, New Price: $100
Name: MASSAGE, Provider: Provider 2, New Price: $100
Name: MASSAGE, Provider: Provider 3, New Price: $100
Name: MASSAGE, Provider: Provider 1, New Price: $100
